<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/Notebook_Pipeline_Credito_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción de Riesgo Crediticio con PySpark y Spark ML

Este cuaderno muestra cómo construir un pipeline de machine learning escalable utilizando PySpark para predecir el riesgo de incumplimiento de crédito. Se utilizará un conjunto de datos abierto disponible en Kaggle, modificado para permitir descarga directa.

El pipeline incluye:
- Transformación de variables categóricas (`StringIndexer`)
- Combinación de características (`VectorAssembler`)
- Escalado de variables (`StandardScaler`)
- Entrenamiento de un modelo de regresión logística (`LogisticRegression`)
- Evaluación con AUC (`BinaryClassificationEvaluator`)

In [1]:
# Instalar PySpark si está usando Google Colab
!pip install pyspark

In [ ]:
# Importación de librerías
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
# Crear sesión de Spark
spark = SparkSession.builder.appName("RiesgoCrediticio").getOrCreate()

### Cargar los datos de riesgo crediticio
Los datos utilizados provienen de una versión simplificada del dataset de default credit clients.
Fuente: [https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset](https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset)

Para propósitos demostrativos, se usa una URL pública alternativa con descarga directa (simulación):

In [ ]:
url = "https://raw.githubusercontent.com/plotly/datasets/master/default-of-credit-card-clients.csv"
df_raw = spark.read.csv(url, header=True, inferSchema=True)
df_raw = df_raw.withColumnRenamed("default payment next month", "label")
df_raw.select("LIMIT_BAL", "AGE", "EDUCATION", "MARRIAGE", "SEX", "label").show(5)

### Preparar el pipeline de análisis
Convertimos variables categóricas y numéricas, las ensamblamos y escalamos para entrenar el modelo.

In [ ]:
indexers = [
    StringIndexer(inputCol=col, outputCol=col+"_index")
    for col in ["EDUCATION", "MARRIAGE", "SEX"]
]
assembler = VectorAssembler(
    inputCols=["LIMIT_BAL", "AGE"] + [col+"_index" for col in ["EDUCATION", "MARRIAGE", "SEX"]],
    outputCol="features_raw")
scaler = StandardScaler(inputCol="features_raw", outputCol="features")
lr = LogisticRegression(featuresCol="features", labelCol="label")
pipeline = Pipeline(stages=indexers + [assembler, scaler, lr])

In [ ]:
# Entrenar el modelo
model = pipeline.fit(df_raw)
predictions = model.transform(df_raw)
predictions.select("label", "prediction", "probability").show(5)

### Evaluar el modelo
Utilizamos el área bajo la curva ROC como métrica de evaluación.

In [ ]:
evaluator = BinaryClassificationEvaluator()
auc = evaluator.evaluate(predictions)
print(f"Área bajo la curva (AUC): {auc:.4f}")